In [ ]:
import emcee
import corner
import speclite as speclite; from speclite import filters
from tqdm import tqdm
from matplotlib import cm
from matplotlib.artist import Artist
from chromatic import *
from scipy.optimize import minimize
from scipy.optimize import curve_fit
from PyAstronomy import pyasl
from specutils.spectra import Spectrum1D, SpectralRegion
from specutils.fitting import fit_generic_continuum


params = {'legend.fontsize': 'x-large',
          'figure.figsize': (6, 4),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)
plt.style.use('tableau-colorblind10')

In [ ]:
def photometric_variability_model(parameters=None,
                                  samples_exist = False,
                                  samples = None,
                                  **kwargs):
    
    f_spot,df_spot,T_spot,T_amb = parameters
    bandpass=np.linspace(3800.,10000.,400)*u.angstrom
    
    #response functions
    
    gp_data = np.loadtxt('../data/filter_curves/SDSS.gp.txt')
    gp_response = gp_data[:,1]
    gp_response[gp_response<0] = 0
    gp_response[-1] = 0
    rp_data = np.loadtxt('../data/filter_curves/SDSS.rp.txt')
    rp_response = rp_data[:,1]
    rp_response[rp_response<0] = 0
    rp_response[-1] = 0
    ip_data = np.loadtxt('../data/filter_curves/SDSS.ip.txt')
    ip_response = ip_data[:,1]
    ip_response[ip_response<0] = 0
    ip_response[-1] = 0
    LCO_gp = speclite.filters.FilterResponse(
        wavelength = gp_data[:,0]*10 * u.Angstrom,
        response = gp_response, meta=dict(group_name='sdss', band_name='gp'))
    LCO_rp = speclite.filters.FilterResponse(
        wavelength = rp_data[:,0]*10 * u.Angstrom,
        response = rp_response, meta=dict(group_name='sdss', band_name='rp'))
    LCO_ip = speclite.filters.FilterResponse(
        wavelength = ip_data[:,0]*10 * u.Angstrom,
        response = ip_response, meta=dict(group_name='sdss', band_name='ip'))
    
    sdss_responses = speclite.filters.load_filters('sdss-gp','sdss-rp','sdss-ip')
    response_g = sdss_responses[0].interpolator(bandpass)
    response_r = sdss_responses[1].interpolator(bandpass)
    response_i = sdss_responses[2].interpolator(bandpass)
    tcs = pyasl.TransmissionCurves()
    tcs.addTESSPassband()
    tc = tcs.getTransCurve("TESS")
    response_TESS = tc(bandpass)

    spotflux = get_phoenix_photons(temperature=int(T_spot), wavelength = bandpass,logg=4.4, metallicity=0.0)[1]
    ambflux = get_phoenix_photons(temperature=int(T_amb), wavelength = bandpass,logg=4.4, metallicity=0.0)[1]
    this_model_spectrum = f_spot*spotflux + (1.0-f_spot)*ambflux
                     
    d_lambda = (bandpass[1]-bandpass[0])
    contrast = 1.-(spotflux/ambflux)
    ds_over_s = -df_spot * ( contrast / ( 1.-f_spot * contrast ) )
    semi_amplitude = np.abs(ds_over_s)

    numerator = np.nansum(semi_amplitude*this_model_spectrum*response_g*d_lambda)
    denominator = np.nansum(this_model_spectrum*response_g*d_lambda)
    modelgp = numerator/denominator

    numerator = np.nansum(semi_amplitude*this_model_spectrum*response_r*d_lambda)
    denominator = np.nansum(this_model_spectrum*response_r*d_lambda)
    modelrp = numerator/denominator

    numerator = np.nansum(semi_amplitude*this_model_spectrum*response_i*d_lambda)
    denominator = np.nansum(this_model_spectrum*response_i*d_lambda)
    modelip = numerator/denominator

    numerator = np.nansum(semi_amplitude*this_model_spectrum*response_TESS*d_lambda)
    denominator = np.nansum(this_model_spectrum*response_TESS*d_lambda)
    modelTESS = numerator/denominator
    
    if visit == 'F21':
        model = np.array([modelgp,modelrp,modelip])
        model_coords = [4750,6200,7550]
        w_err = [500,400,500]
        phot_data = np.array([0.075,0.074,0.05])
        phot_errs = np.array([0.0075,0.0075,0.0075])
        
    if visit == 'S22':
        model = np.array([modelgp,modelrp])
        model_coords = [4750,6200]
        w_err = [500,400]
        phot_data = np.array([0.076,0.076])
        phot_errs = np.array([0.006,0.006])
    
    fig, ax1 = plt.subplots(figsize=(8,5))
    title_label=f'{visit} {order} {modelname}'
    ax1.set_title(title_label,fontsize=16,loc='left')
    ax1.set_xlabel(r'Wavelength $\AA$',fontsize=20)
    ax1.set_ylabel(r'$\frac{\Delta S(\lambda)}{S_{avg}}$',fontsize=22)
    if samples_exist:
        fspot_sam, dfspot_sam, Tspot_sam, Tamb_sam = samples
        sig1_fspot = np.percentile(fspot_sam, [15.9, 50., 84.1]) # central 1-sigma values
        sig1_dfspot = np.percentile(dfspot_sam, [15.9, 50., 84.1])
        sig1_Tspot = np.percentile(Tspot_sam, [15.9, 50., 84.1])
        sig1_Tamb = np.percentile(Tamb_sam, [15.9, 50., 84.1])
        best_params = np.array([sig1_fspot[1], sig1_dfspot[1], sig1_Tspot[1], sig1_Tamb[1]])
        for k in range(0,500):
            i = np.random.randint(low=0,high=(len(Tamb_sam)-1))
            ds_spot = get_phoenix_photons(temperature=Tspot_sam[i], wavelength = bandpass,
                                                           logg=4.4, metallicity=0.0)[1]
            ds_amb = get_phoenix_photons(temperature=Tamb_sam[i], wavelength = bandpass,
                                                           logg=4.4, metallicity=0.0)[1]
            _contrast = 1.-(ds_spot/ds_amb)
            _ds_over_s = -dfspot_sam[i] * ( _contrast / ( 1.-fspot_sam[i] * _contrast ) )
            _semi_amplitude = np.abs(_ds_over_s)
            ax1.plot(bandpass, _semi_amplitude, zorder=0, alpha=0.02, color='deepskyblue')

    ax1.errorbar(model_coords,phot_data,xerr=w_err,yerr=phot_errs,color='k',label='Measured',fmt='none',zorder=1000,linewidth=3)
    ax1.set_xlim(3800,8500)
    ax1.set_ylim(0,0.11)
    ax1.legend(loc='lower right')

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
    ax2.fill_between(bandpass.value, 0, response_g/np.nanmax(response_g),color='orange',
                     zorder=-100,label='SDSS g filter response',alpha=0.3)
    ax2.fill_between(bandpass.value, 0, response_r/np.nanmax(response_r),color='teal',
                     zorder=-100,label='SDSS r filter response',alpha=0.3)
    ax2.fill_between(bandpass.value, 0, response_i/np.nanmax(response_i),color='purple',
                     zorder=-100,label='SDSS i filter response',alpha=0.3)
    ax2.set_ylabel('Filter Response',fontsize=20)
    ax2.set_ylim(0,1.1)

    # if visit == 'combinedwithTESS':
    #     ax1.set_xlim(3800,10000) 
    #     ax2.fill_between(bandpass.value, 0, response_TESS,color='gray',
    #                      zorder=-110,label='TESS fiflter response',alpha=0.2)

    ax2.legend(loc='lower left')
    
    plt.savefig(f'../figs/{label}_photmodel.png',dpi=200)
    plt.show()
    plt.close()

In [ ]:
ordernumbers = np.arange(53,84)
bad_orders = np.array([56,57,59,60,62,63,64,65,66,67,68,69,
                       72,73,74,75,78,79,80,83])
good_orders = np.array([53,54,55,58,61,70,71,76,77,81,82])

# nsteps = 1000
# modeltype = 'Final_Teff_Phot'
# modelname = 'Teff + Phot'
# visits = ['F21','S22']
# orders_I_care_about = [52]

# nsteps = 2000
# modeltype = f'Final_Teff_Phot_Spec'
# modelname = 'Teff + Phot + Spec'
# visits = ['F21','S22']
# orders_I_care_about = good_orders

nsteps = 3000
modeltype = f'Final_Teff_Phot_Spec'
modelname = 'Teff + Phot + Spec'
visits = ['combined']
orders_I_care_about = good_orders

for visit in tqdm(visits):
                
    for order in orders_I_care_about:
        label = f'{visit}_{order}_{modeltype}'
        print(label)
        samples_file_label = label+f'_{nsteps}steps'
        '''
        READ IN THE MCMC SAMPLES
        '''
        reader = emcee.backends.HDFBackend(f'../data/samples/{samples_file_label}.h5')
        sampler = reader.get_chain(discard=int(0.25*nsteps), flat=True)
        samples = sampler.reshape((-1, 4)).T
        fspot_sam, dfspot_sam, Tspot_sam, Tamb_sam = samples
        sig1_fspot = np.percentile(fspot_sam, [15.9, 50., 84.1]) # central 1-sigma values
        sig1_dfspot = np.percentile(dfspot_sam, [15.9, 50., 84.1])
        sig1_Tspot = np.percentile(Tspot_sam, [15.9, 50., 84.1])
        sig1_Tamb = np.percentile(Tamb_sam, [15.9, 50., 84.1])
        # Define the 50th percentile and 1-sigma interval parameter values from the samples
        best_params = np.array([sig1_fspot[1], sig1_dfspot[1], sig1_Tspot[1], sig1_Tamb[1]])
        best_params_err_lower = best_params-[sig1_fspot[0], sig1_dfspot[0], sig1_Tspot[0], sig1_Tamb[0]]
        best_params_err_higher = [sig1_fspot[2], sig1_dfspot[2], sig1_Tspot[2], sig1_Tamb[2]]-best_params
        variable_names = [r'f$_{\rm{spot}}$',r'$\Delta$f$_{\rm{spot}}$',r'T$_{\rm{spot}}$',r'T$_{\rm{amb}}$']
        normie_names = ['fspot','dfspot','Tspot','Tamb'] 
        
        photometric_variability_model(parameters=best_params,visit=visit,
                                      plot=True,samples_exist=True,
                                      samples=samples)